In [8]:
import pandas as pd

# ------------------------------------------------------------
# 1. LISTA REAL DE ARCHIVOS EN TU CARPETA
# ------------------------------------------------------------

archivos = [
    ("Resultados_Admision_2024I.csv",  2024, "I"),
    ("Resultados_Admision_2024II.csv", 2024, "II"),
    ("Resultados_Admision_2025I.csv",  2025, "I"),
    ("Resultados_Admision_2025II.csv", 2025, "II"),
    ("Resultados_Admision_2026I.csv",  2026, "I")
]

dfs = []

mapa_periodo = {"I": 1, "II": 2}

# ------------------------------------------------------------
# 2. PROCESAR ARCHIVO POR ARCHIVO
# ------------------------------------------------------------

for archivo, anio, periodo in archivos:
    print(f"📥 Leyendo: {archivo}")

    df = pd.read_csv(archivo, encoding="utf-8-sig")

    # Asegurar año y periodo
    df["anio"] = anio
    df["periodo"] = periodo

    # Crear orden temporal
    df["orden_examen"] = anio + mapa_periodo.get(periodo, 0) / 10

    dfs.append(df)

# ------------------------------------------------------------
# 3. UNIR TODOS LOS DATAFRAMES
# ------------------------------------------------------------

historico = pd.concat(dfs, ignore_index=True)

# ------------------------------------------------------------
# 4. GUARDAR DATASET FINAL
# ------------------------------------------------------------

historico.to_csv("Dataset_Historico_Admision.csv", index=False, encoding="utf-8-sig")


print("\n✔ Dataset histórico generado con éxito")
print("Total filas:", len(historico))
print("Columnas:", list(historico.columns))


📥 Leyendo: Resultados_Admision_2024I.csv
📥 Leyendo: Resultados_Admision_2024II.csv
📥 Leyendo: Resultados_Admision_2025I.csv
📥 Leyendo: Resultados_Admision_2025II.csv
📥 Leyendo: Resultados_Admision_2026I.csv

✔ Dataset histórico generado con éxito
Total filas: 125884
Columnas: ['MODALIDAD', 'APELLIDOS_Y_NOMBRES', 'ESCUELA_PROFESIONAL', 'PUNTAJE', 'MERITO', 'OBSERVACION', 'anio', 'periodo', 'orden_examen', 'ESCUELA_PROFESIONAL.1']


In [9]:
import pandas as pd

# 1. Cargar el dataset histórico ya generado
df = pd.read_csv("Dataset_Historico_Admision.csv", encoding="utf-8-sig")

print("Columnas actuales:", df.columns.tolist())
print("Filas totales:", len(df))

# 2. Unificar la carrera en una sola columna: CARRERA
# Si ESCUELA_PROFESIONAL.1 existe y no es nula, usarla; si no, usar ESCUELA_PROFESIONAL
if "ESCUELA_PROFESIONAL.1" in df.columns:
    df["CARRERA"] = df["ESCUELA_PROFESIONAL.1"].fillna(df["ESCUELA_PROFESIONAL"])
else:
    df["CARRERA"] = df["ESCUELA_PROFESIONAL"]

# 3. Crear columna INGRESA (0/1)
# Regla: 1 si OBSERVACION contiene algo como "ALCANZ" (ALCANZÓ, ALCANZÃ, ALCANZO, etc.)
df["OBSERVACION"] = df["OBSERVACION"].astype(str)

df["INGRESA"] = df["OBSERVACION"].str.upper().str.contains("ALCANZ", na=False).astype(int)

print("\nDistribución de INGRESA:")
print(df["INGRESA"].value_counts())

# 4. Limpiar columnas que no vamos a usar
# Quitamos nombres de postulantes y columnas duplicadas
columnas_a_eliminar = []

if "APELLIDOS_Y_NOMBRES" in df.columns:
    columnas_a_eliminar.append("APELLIDOS_Y_NOMBRES")
if "ESCUELA_PROFESIONAL" in df.columns:
    columnas_a_eliminar.append("ESCUELA_PROFESIONAL")
if "ESCUELA_PROFESIONAL.1" in df.columns:
    columnas_a_eliminar.append("ESCUELA_PROFESIONAL.1")

df = df.drop(columns=columnas_a_eliminar)

# 5. Asegurar que PUNTAJE sea numérico
df["PUNTAJE"] = pd.to_numeric(df["PUNTAJE"], errors="coerce")

# Opcional: quitar filas sin puntaje
df = df.dropna(subset=["PUNTAJE"])

print("\nFilas luego de limpiar puntaje:", len(df))

# 6. Definir variables para el modelo
# Variables numéricas y categóricas que usaremos
features_numericas = ["PUNTAJE", "anio", "orden_examen"]
features_categoricas = ["MODALIDAD", "CARRERA", "periodo"]

X = df[features_numericas + features_categoricas]
y = df["INGRESA"]

# 7. Separar train/test
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# 8. Preprocesamiento: OneHot para categóricas
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier

preprocesador = ColumnTransformer(
    transformers=[
        ("num", "passthrough", features_numericas),
        ("cat", OneHotEncoder(handle_unknown="ignore"), features_categoricas),
    ]
)

# 9. Definir modelo (Random Forest)
modelo = RandomForestClassifier(
    n_estimators=200,
    max_depth=None,
    random_state=42,
    n_jobs=-1
)

# 10. Crear pipeline: preprocesamiento + modelo
clf = Pipeline(steps=[
    ("preprocesador", preprocesador),
    ("modelo", modelo)
])

# 11. Entrenar
clf.fit(X_train, y_train)

# 12. Evaluar
from sklearn.metrics import classification_report, confusion_matrix

y_pred = clf.predict(X_test)

print("\nMATRIZ DE CONFUSIÓN:")
print(confusion_matrix(y_test, y_pred))

print("\nREPORTE DE CLASIFICACIÓN:")
print(classification_report(y_test, y_pred, digits=4))

# 13. Probar una predicción con un ejemplo
ejemplo = pd.DataFrame([{
    "PUNTAJE": 900.0,
    "anio": 2026,
    "orden_examen": 2026.1,
    "MODALIDAD": "EDUCACIÓN BÁSICA REGULAR (EBR) Y EDUCACIÓN BÁSICA ALTERNATIVA (EBA)",
    "CARRERA": "MEDICINA HUMANA",
    "periodo": "I"
}])

prob_ingresa = clf.predict_proba(ejemplo)[0][1]
print("\nProbabilidad de ingreso para el ejemplo:", prob_ingresa)

# 14. (Opcional) Guardar modelo entrenado
import joblib
joblib.dump(clf, "modelo_ingreso_unmsm.pkl")
print("\n✔ Modelo guardado como 'modelo_ingreso_unmsm.pkl'")


C:\Users\alejo\AppData\Local\Temp\ipykernel_21768\3865001324.py:4: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("Dataset_Historico_Admision.csv", encoding="utf-8-sig")


Columnas actuales: ['MODALIDAD', 'APELLIDOS_Y_NOMBRES', 'ESCUELA_PROFESIONAL', 'PUNTAJE', 'MERITO', 'OBSERVACION', 'anio', 'periodo', 'orden_examen', 'ESCUELA_PROFESIONAL.1']
Filas totales: 125884

Distribución de INGRESA:
INGRESA
0    111767
1     14117
Name: count, dtype: int64

Filas luego de limpiar puntaje: 124262

MATRIZ DE CONFUSIÓN:
[[22042    42]
 [   61  2708]]

REPORTE DE CLASIFICACIÓN:
              precision    recall  f1-score   support

           0     0.9972    0.9981    0.9977     22084
           1     0.9847    0.9780    0.9813      2769

    accuracy                         0.9959     24853
   macro avg     0.9910    0.9880    0.9895     24853
weighted avg     0.9958    0.9959    0.9958     24853


Probabilidad de ingreso para el ejemplo: 0.0

✔ Modelo guardado como 'modelo_ingreso_unmsm.pkl'


In [10]:
import pandas as pd

# 1. Cargar dataset histórico
df = pd.read_csv("Dataset_Historico_Admision.csv", encoding="utf-8-sig")

# 2. Crear columna INGRESA (1 si observación contiene 'ALCANZ')
df["OBSERVACION"] = df["OBSERVACION"].astype(str)

df["INGRESA"] = df["OBSERVACION"].str.upper().str.contains("ALCANZ", na=False).astype(int)

print("Distribución INGRESA (0 = no, 1 = sí):")
print(df["INGRESA"].value_counts())

# 3. Normalizar el nombre para agrupar postulantes
df["nombre_norm"] = df["APELLIDOS_Y_NOMBRES"].str.upper().str.strip()

# 4. Contar cuántas postulaciones tiene cada persona (en toda la historia)
postulaciones_por_persona = (
    df.groupby("nombre_norm")
      .size()                      # número de filas por persona
      .rename("n_postulaciones")
)

# 5. Identificar quiénes ALGUNA VEZ ingresan
nombres_ingresantes = df.loc[df["INGRESA"] == 1, "nombre_norm"].unique()

# Filtrar solo esos en la serie de postulaciones
postulaciones_ingresantes = postulaciones_por_persona.loc[nombres_ingresantes]

# 6. Ver distribución: cuántas veces postulan los que llegan a ingresar
dist_postulaciones = postulaciones_ingresantes.value_counts().sort_index()

print("\n📊 Veces que postulan los que alguna vez ingresan:")
print(dist_postulaciones)

# 7. (Opcional) Ver algunos ejemplos
print("\nEjemplos de ingresantes que postularon más de 1 vez:")
print(postulaciones_ingresantes[postulaciones_ingresantes > 1].sort_values(ascending=False).head(20))


Distribución INGRESA (0 = no, 1 = sí):
INGRESA
0    111767
1     14117
Name: count, dtype: int64

📊 Veces que postulan los que alguna vez ingresan:
n_postulaciones
1    5835
2    4634
3    2222
4     989
5     154
Name: count, dtype: int64

Ejemplos de ingresantes que postularon más de 1 vez:
nombre_norm
MEJIA MALLQUI, MEDALY ELIZABETH              5
CUYA DURAND, ANAHI NAOMI                     5
ARANDA ESPINOZA, LUIS ANGEL                  5
LLALLAHUI TANTA, ANTHONY                     5
SANTIAGO ULLERO, LEONARDO EXSUNEL RAPHAEL    5
HUANEY BONIFACIO, DIONY MARK                 5
BARRETO DOMÍNGUEZ, DEINA LUZ                 5
FERNANDEZ TAMARA, ALDO ALEXANDER             5
SUAREZ YAMUNAQUE, ADRIAN MARTIN              5
CHAMBILLA POMA, SUMMY DEYSI                  5
MORETO SUAREZ, RUBEN OMAR                    5
VILLEGAS RUIZ, DEYSSI ESTRELLA               5
CUADROS CLARO, CANDY DEL ROSARIO             5
GUARDIA SILUPU, JIMMY FERNANDO               5
MEDRANO MALLQUI, ANGELINA SUMMY     

C:\Users\alejo\AppData\Local\Temp\ipykernel_21768\627827131.py:4: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("Dataset_Historico_Admision.csv", encoding="utf-8-sig")


In [11]:
import pandas as pd

def predecir_ingreso(puntaje, modalidad, carrera, anio, periodo="I"):
    """
    Devuelve la probabilidad de ingreso según tu modelo
    usando como input el puntaje y datos del examen.
    """
    # calculamos orden_examen igual que en tu histórico
    mapa_periodo = {"I": 1, "II": 2}
    orden_examen = anio + mapa_periodo.get(periodo, 0) / 10.0

    ejemplo = pd.DataFrame([{
        "PUNTAJE": puntaje,
        "anio": anio,
        "orden_examen": orden_examen,
        "MODALIDAD": modalidad,
        "CARRERA": carrera,
        "periodo": periodo
    }])

    prob = clf.predict_proba(ejemplo)[0][1]  # probabilidad de INGRESA=1
    return prob


In [17]:
mi_puntaje_ultimo = 1520.0
mi_modalidad = "EDUCACIÓN BÁSICA REGULAR (EBR) Y EDUCACIÓN BÁSICA ALTERNATIVA (EBA)"
mi_carrera = "MEDICINA HUMANA"
anio_siguiente_examen = 2026
periodo_examen = "I"

prob = predecir_ingreso(mi_puntaje_ultimo, mi_modalidad, mi_carrera, anio_siguiente_examen, periodo_examen)
print("Probabilidad de ingresar:", prob)


Probabilidad de ingresar: 1.0
